# Predict for each patient

In [1]:
import pandas as pd

from data import load_data

train_data, additional_train_data, test_data, unique_patients = load_data()

In [2]:
from pipelines import pipeline

data = {}
for p_num in unique_patients:
    train_data_patient = pd.concat([train_data[train_data['p_num'] == p_num], additional_train_data[additional_train_data['p_num'] == p_num]])
    test_data_patient = test_data[test_data['p_num'] == p_num]

    train_data_patient_transformed = pipeline.fit_transform(train_data_patient)
    test_data_patient_transformed = pipeline.transform(test_data_patient)

    data[p_num] = {
        'X_train': train_data_patient_transformed.drop(columns=['bg+1:00']),
        'y_train': train_data_patient_transformed['bg+1:00'],
        'X_test': test_data_patient_transformed,
        'y_pred': test_data_patient_transformed[[]]
    }

In [3]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import StackingRegressor
import os
import datetime
from regressors import get_hgb_regressor, get_xgb_regressor, get_knn_regressor, get_keras_regressor, get_lasso_lars_ic_regressor


def get_date_time_now():
    return datetime.datetime.now().strftime('%H:%M:%S')


models = {}
results = test_data[[]].copy()
results.loc[:, 'bg+1:00'] = -1.0
results['bg+1:00'] = results['bg+1:00'].astype(float)
for p_num in data.keys():
    print(f'{get_date_time_now()} - {p_num} - Predicting for patient {p_num}')
    print(f'{get_date_time_now()} - {p_num} - Transforming the data -')

    X_train = data[p_num]['X_train']
    y_train = data[p_num]['y_train']
    X_test = data[p_num]['X_test']

    print(f'{get_date_time_now()} - {p_num} - Fitting the model -')

    _, X_val, _, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

    estimators = [
        ('hgb', get_hgb_regressor()),
        ('xgb', get_xgb_regressor()),
        ('llic', get_lasso_lars_ic_regressor()),
        ('knn', get_knn_regressor()),
        ('dnn', get_keras_regressor(
            p_num=p_num,
            X_train=X_train,
            X_val=X_val,
            y_train=y_train,
            y_val=y_val))
    ]

    model = StackingRegressor(estimators=estimators, final_estimator=Ridge(alpha=0.1), n_jobs=1, verbose=2)
    model.fit(X=X_train, y=y_train)

    print(f'{get_date_time_now()} - {p_num} - Predicting -')
    y_pred = model.predict(X=X_test)

    # count and replace negative values
    if np.sum(y_pred < 0) > 0:
        print(f'{get_date_time_now()} - {p_num} - Number of negative values: {np.sum(y_pred < 0)}')
        bg_min_train = np.min(y_train)
        print(f'{get_date_time_now()} - {p_num} - Min value: {np.min(y_pred)}')
        y_pred = np.where(y_pred < 0, bg_min_train, y_pred)

    data[p_num]['y_pred'].loc[data[p_num]['X_test'].index, 'bg+1:00'] = y_pred
    results.loc[data[p_num]['X_test'].index, 'bg+1:00'] = y_pred
    print(f'{get_date_time_now()} - {p_num} - Done -')

results.head()

00:36:31 - p01 - Predicting for patient p01
00:36:31 - p01 - Transforming the data -
00:36:31 - p01 - Fitting the model -
Epoch 1/100
601/601 - 1s - 2ms/step - loss: 42.5974 - rmse: 6.1399 - val_loss: 7.9503 - val_rmse: 2.7899
Epoch 2/100
601/601 - 0s - 697us/step - loss: 11.5053 - rmse: 3.3540 - val_loss: 7.3328 - val_rmse: 2.6820
Epoch 3/100
601/601 - 0s - 696us/step - loss: 10.1646 - rmse: 3.1554 - val_loss: 6.9792 - val_rmse: 2.6109
Epoch 4/100
601/601 - 0s - 696us/step - loss: 9.2451 - rmse: 3.0091 - val_loss: 6.8161 - val_rmse: 2.5828
Epoch 5/100
601/601 - 0s - 694us/step - loss: 9.0382 - rmse: 2.9771 - val_loss: 6.7983 - val_rmse: 2.5805
Epoch 6/100
601/601 - 0s - 698us/step - loss: 8.6381 - rmse: 2.9073 - val_loss: 6.6432 - val_rmse: 2.5324
Epoch 7/100
601/601 - 0s - 680us/step - loss: 8.4301 - rmse: 2.8741 - val_loss: 6.7989 - val_rmse: 2.5672
Epoch 8/100
601/601 - 0s - 684us/step - loss: 8.2862 - rmse: 2.8456 - val_loss: 6.5107 - val_rmse: 2.5194
Epoch 9/100
601/601 - 0s - 70

/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


601/601 - 0s - 640us/step - loss: 5.9932 - rmse: 4.8522


/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


481/481 - 0s - 758us/step - loss: 5.7490 - rmse: 4.7963
121/121 - 0s - 912us/step


...

311/311 - 0s - 843us/step - loss: 2.3433 - rmse: 2.9802
78/78 - 0s - 1ms/step


/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


311/311 - 0s - 834us/step - loss: 2.5065 - rmse: 2.9728
78/78 - 0s - 1ms/step
01:11:41 - p24 - Predicting -
9/9 - 0s - 8ms/step
01:11:41 - p24 - Done -


/var/folders/g7/z7m625rj50d5ptztzplh4wgc0000gn/T/ipykernel_44209/4056389323.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[p_num]['y_pred'].loc[data[p_num]['X_test'].index, 'bg+1:00'] = y_pred


,bg+1:00
id,
p01_8459,8.476643
p01_8460,7.035729
p01_8461,11.247608
p01_8462,11.681006
p01_8463,8.078189


In [4]:
results[results['bg+1:00'] < 0]

,bg+1:00
id,


In [5]:
# check if all results are greater than 0
assert (results['bg+1:00'] >= 0).all()

## Prepare the submission file

In [6]:
submission = results
submission

,bg+1:00
id,
p01_8459,8.476643
p01_8460,7.035729
p01_8461,11.247608
p01_8462,11.681006
p01_8463,8.078189
...,...
p24_256,6.765179
p24_257,9.281958
p24_258,6.683356


### Save the submission file

In [7]:
submission.to_csv(f'submission-{os.path.basename(os.getcwd())}.csv')

In [8]:
train_data['bg+1:00'].describe()

count    150996.000000
mean          8.224292
std           2.967680
min           2.200000
25%           6.100000
50%           7.600000
75%           9.800000
max          27.800000
Name: bg+1:00, dtype: float64

In [9]:
train_data[train_data['p_num'] == 'p16']['bg+1:00'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: bg+1:00, dtype: float64